In [12]:
from pyspark.sql import SparkSession, DataFrame
import requests
import pandas as pd
import pickle
import os

In [13]:
def salvar_picke(objeto, caminho_arquivo: str):
    with open(caminho_arquivo, 'wb') as arquivo:
        pickle.dump(objeto, arquivo)

In [14]:
spark = SparkSession.builder.appName('DEPARA').getOrCreate()

- Depara Trends

In [4]:
df_trends = spark.read.parquet('/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/ouro/trends_brazil/trends_brazil/trends_brazil.parquet')

ERROR:root:KeyboardInterrupt while sending command.                             
Traceback (most recent call last):
  File "/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/venv/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
df_trends.show()

+-------------------------+-------------------------+---------------------+-------------------+---------------+-----------------+-----------+-------------------------+---------------------+-----------------------+-----------------+-------------+------------+--------------------+-----------+--------------+
|                 NM_CANAL|             TITULO_VIDEO|INDICE_TURNO_EXTRACAO|TOTAL_VISUALIZACOES|TOTAL_FAVORITOS|TOTAL_COMENTARIOS|TOTAL_LIKES|TOTAL_VISUALIZACOES_TURNO|TOTAL_FAVORITOS_TURNO|TOTAL_COMENTARIOS_TURNO|TOTAL_LIKES_TURNO|data_extracao|ID_CATEGORIA|            ID_CANAL|   ID_VIDEO|TURNO_EXTRACAO|
+-------------------------+-------------------------+---------------------+-------------------+---------------+-----------------+-----------+-------------------------+---------------------+-----------------------+-----------------+-------------+------------+--------------------+-----------+--------------+
|     Melhor Futebol do...|     REAL SOCIEDAD VEN...|                    1|    

In [ ]:
params = {
    'part': 'snippet',
    'regionCode': 'BR',
    'key' : ''
}

In [22]:
req = requests.get('https://youtube.googleapis.com/youtube/v3/videoCategories', params=params)

In [23]:
dados = []

In [24]:

for item in req.json()['items']:
    dado = {
        'ID' : item['id'],
        'NOME_CATEGORIA': item['snippet']['title']
    }
    dados.append(dado)

In [30]:
df_categoria = pd.DataFrame(dados)
salvar_picke(df_categoria,'../depara/categoria.pkl', )

In [65]:
df_depara_canal_trends = df_trends.select('ID_CANAL', 'NM_CANAL').distinct()
df_depara_canal_trends.show()

+--------------------+-------------------------------+
|            ID_CANAL|                       NM_CANAL|
+--------------------+-------------------------------+
|UC4aiJNDUviw_vMhd...|           Cortes do Casimit...|
|UCX8pfKRW496XSHsU...|           Vem a mim língua ...|
|UCiQ7Ll2Gsb5eQhgQ...|           Melhor Futebol do...|
|UCritGVo7pLJLUS8w...|      (G)I-DLE (여자)아이들 ...|
|UC407XQEh4grFPh-F...|           CANAL DO NILSON C...|
|UCIv5-lGsVuW3JKPG...|           Laerte Viana na á...|
|UCgLuBHpgnAXKKq3u...|           Fire Player [Atai...|
|UCJ-_JtS_-0G7pt6C...|           Família Jeito lok...|
|UCAAPXtnzlg9krw6M...|           Fluminense Footba...|
|UCv-Nx8pSfG_LxbVi...|             Jovem Pan Esportes|
|UCs-6sCz2LJm1PrWQ...|              TNT Sports Brasil|
|UCLhgIX2L5ZCaWdla...|東映アニメーション公式YouTub...|
|UCTNFtLQWA94BNeu0...|                Isamara Amâncio|
|UCc0cU92Tf7rvd2gr...|                Opinião Sincera|
|UCP_WlkTCG-e4ULEA...|             Letícia Sweet Cake|
|UCyJ-fWKmF40BQ1FN...|    

In [66]:
df_depara_canal_trends = df_depara_canal_trends.collect()

In [68]:
canal_trends = [{'label': linha['ID_CANAL'], 'value': linha['NM_CANAL']}for linha in df_depara_canal_trends]


In [70]:
salvar_picke(canal_trends, '../depara/trends/canal_trends.pkl')

In [71]:
df_data_extracao = df_trends.select('data_extracao').distinct()
df_data_extracao.show()

+-------------+
|data_extracao|
+-------------+
|   2023-10-25|
|   2023-10-15|
|   2023-10-27|
|   2023-10-23|
|   2023-10-21|
|   2023-10-20|
|   2023-10-19|
|   2023-10-22|
|   2023-10-26|
|   2023-10-24|
|   2023-10-16|
+-------------+



In [72]:
lista_datas = df_data_extracao.toPandas()['data_extracao'].to_list()

In [73]:
salvar_picke(lista_datas, '../depara/trends/lista_datas.pkl')

In [77]:
lista_video_trends = df_trends.select('ID_VIDEO', 'TITULO_VIDEO').distinct().collect()
lista_video_trends

[Row(ID_VIDEO='Iewisp9KYRw', TITULO_VIDEO="(여자)아이들((G)I-DLE) - 'I Want That' Official Music Video"),
 Row(ID_VIDEO='05yeNSE8jDc', TITULO_VIDEO='Cheguei em Moscou 🇷🇺 | Vlog viajando para Rússia pela primeira vez! Voo, hotel e cidade incrível #1'),
 Row(ID_VIDEO='MHQXhXJdQB4', TITULO_VIDEO='CASIMIRO REAGE: URUGUAI 2 X 0 BRASIL - ELIMINATÓRIAS PARA A COPA (DESESPERADOR) | Cortes do Casimito'),
 Row(ID_VIDEO='j4TlIePiewE', TITULO_VIDEO='REAL SOCIEDAD VENCE E AFUNDA O BENFICA NO GRUPO D! BENFICA 0 X 1 REAL SOCIEDAD | CHAMPIONS'),
 Row(ID_VIDEO='1EEZtNFDU8Q', TITULO_VIDEO='NILSON CESAR DETONA SELEÇÃO BRASILEIRA! | Cortes Rádio de Pai pra Filho'),
 Row(ID_VIDEO='ixxOASickcs', TITULO_VIDEO='A BRUX4 TÁ SOLTA! ENTENDA O PORQUÊ PAULO BORRACHINHA ESTÁ FORA DA LUTA CONTRA CHIMAEV NO #UFC294'),
 Row(ID_VIDEO='ouVWU9Svk1U', TITULO_VIDEO='Chegamos na Itália 🇮🇹 NOSSO NOVO APARTAMENTO + PRIMEIRA COMPRA NO MERCADO EM EURO💶 muito barato😱'),
 Row(ID_VIDEO='uK59gQ9JyxQ', TITULO_VIDEO='FLUTV - BASTIDORES - 

In [81]:
video_trends = [{'label': linha['TITULO_VIDEO'], 'value': linha['ID_VIDEO']}for linha in lista_video_trends]
video_trends
salvar_picke(video_trends, '../depara/trends/video_trends.pkl')

In [16]:
df_canal = spark.read.parquet('/home/rodrigo/Documentos/projetos/open_weather_api_apache/analise_dados_youtube/data/projetos_youtube_v2/novo_ouro/total_visualizacoes_semana.parquet')
df_canal.show()

+---------------+--------------------+---------------------+--------------------+---------------------+----------+-------------------+-------------------------+-----------------+-----------------------+-----------+-----------------+-------------+--------------------+--------------------+-----------+--------------+
|       NM_CANAL|        TITULO_VIDEO|TOTAL_CARACTERE_VIDEO|                TAGS|DURACAO_VIDEO_MINUTOS|TOTAL_TAGS|TOTAL_VISUALIZACOES|TOTAL_VISUALIZACOES_TURNO|TOTAL_COMENTARIOS|TOTAL_COMENTARIOS_TURNO|TOTAL_LIKES|TOTAL_LIKES_TURNO|data_extracao|             ASSUNTO|            ID_CANAL|   ID_VIDEO|TURNO_EXTRACAO|
+---------------+--------------------+---------------------+--------------------+---------------------+----------+-------------------+-------------------------+-----------------+-----------------------+-----------+-----------------+-------------+--------------------+--------------------+-----------+--------------+
|ChratosGameplay|CITIES SKYLINES 2...|              

In [59]:
df_canal.select('ASSUNTO').distinct().show(truncate=False)

+-------------------------+
|ASSUNTO                  |
+-------------------------+
|assunto_cities_skylines_2|
|assunto_power_bi         |
|assunto_cities_skylines  |
|assunto_python_and_dados |
+-------------------------+



In [60]:
lista_de_assuntos = ['assunto_cities_skylines_2', 'assunto_power_bi', 'assunto_cities_skylines', 'assunto_python_and_dados']

In [110]:

lista_canais_depara = []
for assunto in lista_de_assuntos:
    dicionario_depara_canal = {}
    df_canal_depara = df_canal.select('ASSUNTO', 'ID_CANAL', 'NM_CANAL').filter(df_canal.ASSUNTO == assunto).distinct().collect()
    lista_canal_depara = [
        {
            'label': linha['NM_CANAL'], 
            'value': linha['ID_CANAL']
        } for linha in df_canal_depara
    ]

    dicionario_depara_canal['ASSUNTO'] = assunto
    dicionario_depara_canal['VALORES'] = lista_canal_depara
    print(dicionario_depara_canal)
    lista_canais_depara.append(
        dicionario_depara_canal
    )


{'ASSUNTO': 'assunto_cities_skylines_2', 'VALORES': [{'label': 'Majestic', 'value': 'UCxIMGC-GmZ2pXE_jGGW2C9A'}, {'label': 'MetalBear', 'value': 'UCfY4ggUDPeecGSCNePyyVSA'}, {'label': 'Canal do Void', 'value': 'UCrUcdH_bCfX77xpXHyq2fJw'}, {'label': 'Canal Sala de Jogos', 'value': 'UCYEb5mbKZ5JJzN0xBuKyaig'}, {'label': 'SuinandoJogos', 'value': 'UC3P9lC-O8Rr1Ggc3oIIqIkg'}, {'label': 'Irmãos Cities', 'value': 'UC1mk6EtfMjxR4eEZ7C43zTQ'}, {'label': 'Cortes Boladao', 'value': 'UCYDluKEjFODZQueXT0pZJfg'}, {'label': 'JulioGameplay', 'value': 'UCL5k8vBBo1JOHh7AnvoD8xQ'}, {'label': 'Ganso GamePlay', 'value': 'UCp7gzkfbWE7jazP65W9ZtWg'}, {'label': 'Barbosa Cities', 'value': 'UCe9jrI0YQ5SM6h5QRZ9FZlA'}, {'label': 'Wellington Souza', 'value': 'UCBSrzvbMoru6PsayN3Z7ZsA'}, {'label': 'Simularies', 'value': 'UCDGo_s0I-2JpO3o3xlkMQVA'}, {'label': 'Lives do IVDMALUCO', 'value': 'UCn7w7H6jL0G3TNpV1v0SZRg'}, {'label': 'Lives do Chratos', 'value': 'UCayiH2Y5JBEOSubqoQhufDA'}, {'label': 'ChratosGameplay', 

In [116]:
salvar_picke(lista_canais_depara, '../src/depara/outros/lista_canais_depara.pkl')

In [124]:

lista_video_depara = []
for assunto in lista_de_assuntos:
    dicionario_depara_video = {}
    df_canal_depara = df_canal.select('ASSUNTO', 'TITULO_VIDEO', 'ID_VIDEO').filter(df_canal.ASSUNTO == assunto).distinct().collect()
    lista_depara = [
        {
         'label': linha['TITULO_VIDEO'], 
         'value': linha['ID_VIDEO']} for linha in df_canal_depara]
 
    dicionario_depara_video['ASSUNTO'] = assunto
    dicionario_depara_video['VALORES'] = lista_depara
    print(dicionario_depara_video)
    lista_video_depara.append(
        dicionario_depara_video
    )

{'ASSUNTO': 'assunto_cities_skylines_2', 'VALORES': [{'label': 'PRIMEIRA MEGA EXPANÇÃO POPULACIONAL E INDUSTRIAL #3 CITIES SKYLINES 2 do 0 ao 1 milhão de habitantes', 'value': 'Si87tevdXuc'}, {'label': 'Reduza os lags e travamentos no cities skylines 2  Tutorial #shorts #citiesskylines2', 'value': 'Gg-yu0RVZMY'}, {'label': '🚈 EXPANSÃO FERROVIÁRIA e NÍVEL 7 da CIDADE 🌆 Cities Skylines 2 | Gameplay PT BR ⭐', 'value': 'O37hZJz5pjY'}, {'label': 'CITIES SKYLINES 2 RODANDO QUASE FRITANDO O PC [Gameplay PT-BR]', 'value': '_xXFIIB6UGY'}, {'label': '🏙️ PRIMEIRO BAIRRO INDUSTRIAL 🏗️ Cities Skylines 2 | Gameplay PT BR ⭐', 'value': 'C1s-hsI9JaY'}, {'label': 'Cities Skylines pegamos uma empréstimo e construímos novas zonas residenciais e industriais', 'value': 'D3IsaJDB5ew'}, {'label': 'População Recebe Serviços Funerário - Cities Skylines 2 #citiesskylines2 #citiesskylines', 'value': 'E2KEJU765SQ'}, {'label': 'Expansão em massa! Precisamos de mais gente!!! | Cities Skylines 2 #08', 'value': 'g-MxT

In [132]:
lista_video_depara[2]

{'ASSUNTO': 'assunto_cities_skylines',
 'VALORES': [{'label': '🏙️ PRIMEIRO BAIRRO INDUSTRIAL 🏗️ Cities Skylines 2 | Gameplay PT BR ⭐',
   'value': 'C1s-hsI9JaY'},
  {'label': 'Indústria, Trens e muita conta pra pagar | Cities Skylines 2 PT-BR',
   'value': 'KsWGb7aVhkI'},
  {'label': 'ESTÁDIO Moderno (ou seria uma ARENA?) | CORNELIA - EP19 | Cities: Skylines GAMEPLAY PT-BR',
   'value': 'gYl77Ot9oDU'},
  {'label': 'Cities Skylines uma nova cidade começando do zero vida de prefeito  ep 1',
   'value': 'nMMr_myn5yI'},
  {'label': 'AGUARDANDO  cities skylines2', 'value': 'WxbGu6RTIWg'},
  {'label': 'A histórica construção de Metal York | Cities Skylines 2 PT-BR',
   'value': 'v3YF3Bgyq68'},
  {'label': 'Cities Skylines, ÚLTIMAS PRAIAS!! - Mountain Valley - EP 62 (SEM MODS)',
   'value': '2c5QZgVUdGo'},
  {'label': 'CITIES SKYLINES 2, expansão da cidade!!! EP 02 - Irmãos Cities',
   'value': 'qBsTx6SbuEI'},
  {'label': 'Cities Skylines, RECONSTRUINDO A CIDADE DO CAOS!!',
   'value': 'upq6K

In [133]:
salvar_picke(lista_video_depara, '../src/depara/outros/lista_video_depara.pkl')    

In [11]:
spark.stop()